In [ ]:
import os
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

folder_path = './csv'  # Replace with the actual folder path

# Get a list of all CSV files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# Create an empty list to store the data from each CSV file
data = []
dataset = pd.DataFrame(pd.read_csv('dataset.csv'))
# Iterate over each CSV file
# for i,file in enumerate(csv_files):
#     file_path = os.path.join(folder_path, file)
    
#     # Read the CSV file and append its data to the list
#     chunk = pd.DataFrame(pd.read_csv(file_path))
#     chunk.dropna(inplace=True,subset=['label'])
#     filtered_chunk = chunk[~chunk['label'].isin(['unknown'])]
#     # Append the filtered chunk to the list
#     data.append(filtered_chunk)
#     print("File read : ",i/len(csv_files)*100,"%")
   
# # Concatenate the data from all CSV files into a single dataset
# dataset = pd.concat(data, ignore_index=True)

# dataset.to_csv('dataset.csv', index=False)

# Get the 'label' column from the dataset
output = dataset['label']
# Reshape the output to a 2D array
output = output.values.reshape(-1, 1)
# Create an instance of OneHotEncoder
encoder = OneHotEncoder()
# Fit and transform the output using OneHotEncoder
output_encoded = encoder.fit_transform(output)


# Separate the one-hot encoded "label" column as input and the rest as output
output_data = output_encoded
input_data = dataset.drop('label', axis=1)

print(output_data)


In [30]:
import keras
import numpy as np
from keras import layers
from sklearn.model_selection import KFold

# Define the MLP model architecture
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(input_data.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(output_data.shape[1], activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Perform K-fold cross-validation
k = 5  # Number of folds
kf = KFold(n_splits=k, shuffle=True)

# Initialize lists to store the evaluation results
loss_scores = []
accuracy_scores = []

# Iterate over each fold
for train_index, test_index in kf.split(input_data):
    # Split the data into training and test sets
    X_train, X_test = input_data[train_index], input_data[test_index]
    y_train, y_test = output_data[train_index], output_data[test_index]

    # Train the model on the training set
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

    # Evaluate the model on the test set
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

    # Store the evaluation results
    loss_scores.append(loss)
    accuracy_scores.append(accuracy)

# Calculate the average evaluation scores
avg_loss = np.mean(loss_scores)
avg_accuracy = np.mean(accuracy_scores)

print("Average Loss:", avg_loss)
print("Average Accuracy:", avg_accuracy)

TypeError: unhashable type: 'list'